In [ ]:
WORKING_DIR = "/home/xavier/Documents/DAE_project"

# Reconstruct images with different hyper parameters

In [1]:
import pandas as pd
import os
import shutil

IMG_DIR = f"{WORKING_DIR}/dataset/Roy_training/images"
container = []

for strain in os.listdir(IMG_DIR):
    run_id = int(strain.split("Run")[1])
    strain_path = os.path.join(IMG_DIR, strain)
    for scope in os.listdir(strain_path):
        scope_path = os.path.join(strain_path, scope)
        container.append([run_id, int(scope[-2:]), scope_path])
container = pd.DataFrame(container, columns=["Run", "Scope", "Directory"])

LABEL_DIR = f'{WORKING_DIR}/dataset/Roy_training/Caro 3d 9.7.22_2.20_new.xlsx'

label_df = pd.read_excel(LABEL_DIR)

annotated_container = pd.merge(container, label_df, on="Run")

OUT_DIR = f"{WORKING_DIR}/images/sup_figure2"
selected_dir = os.path.join(OUT_DIR, "targets")
os.makedirs(selected_dir, exist_ok=True)
SELECTED_FRAMES = [1441]
selected_strains = [1622, 8615, 2232, 4398, 4299, 5208, 4408]
filtered_df = annotated_container[annotated_container["Mutant #"].isin(selected_strains)]
for _, row in filtered_df.iterrows():
    images = os.listdir(row["Directory"])
    name_format = images[0][:-8] + "%04d.jpg"
    for selected_frame in SELECTED_FRAMES:
        try:
            shutil.copy(os.path.join(row["Directory"], name_format % selected_frame),
                        os.path.join(selected_dir, f"{row['Mutant #']}_" + name_format % selected_frame))
        except FileNotFoundError:
            print(f"{selected_frame} not found in {row['Directory']}")

1441 not found in D:/Projects/trainingset2\CS2_44_1622_1%agar_Run0320\Scope22
1441 not found in D:/Projects/trainingset2\CS2_44_1622_1%agar_Run0320\Scope23
1441 not found in D:/Projects/trainingset2\CS2_44_1622_1%agar_Run0320\Scope24


In [2]:
# Helper functions
import os
import cv2
import numpy as np
import pandas as pd
import torch
import dnnlib
import legacy
from multiprocessing import Pool
import functools
from training.networks_stylegan2 import SynthesisLayer, MinibatchStdLayer

os.environ['CC'] = "/usr/bin/gcc-9"
os.environ['CXX'] = "/usr/bin/g++-9"

device = torch.device('cuda')


def reset_noise_const(G, seed):
    torch.manual_seed(seed)

    for block in G.synthesis.children():
        for layer in block.children():
            if layer.__class__.__name__ == "SynthesisLayer":  #isinstance(layer, SynthesisLayer):
                resolution = layer.resolution

                with torch.no_grad():
                    layer.noise_const.copy_(torch.randn([resolution, resolution]))


def resize_crop(img_name, resize_by=1., resolution=512, brightness_norm=True, brightness_mean=107):
    img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)
    if img.dtype != np.uint8:
        img = np.uint8(img / 256)
    img_shape = img.shape
    resize_shape = np.array([img_shape[1] * resize_by, img_shape[0] * resize_by], dtype=int)
    if resize_by != 1:
        img = cv2.resize(img, resize_shape, cv2.INTER_LANCZOS4)
    img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2,
    (resize_shape[0] - resolution) // 2:(resize_shape[0] + resolution) // 2]
    if brightness_norm:
        obj_v = np.mean(img)
        value = brightness_mean - obj_v
        img = cv2.add(img, value)
    return img


target_images = []
img_names = []
target_folder = os.path.join(OUT_DIR, "reconstructions/targets")
os.makedirs(target_folder, exist_ok=True)
for img_name in os.listdir(os.path.join(OUT_DIR, "selected_targets")):
    target_image = resize_crop(os.path.join(OUT_DIR, "selected_targets", img_name))
    target_images.append(target_image)
    img_names.append(img_name)
    cv2.imwrite(os.path.join(target_folder, img_name), target_image)

# row1 = np.hstack(target_images[:4])  # First row
# row2 = np.hstack(target_images[4:8])  # Second row
# grid_image = np.vstack([row1, row2])  # Stack rows
# cv2.imwrite(os.path.join(OUT_DIR, "reconstructions_none", f"target.png"), grid_image)
target_images = np.array(target_images)
target_images = torch.Tensor(target_images).to(device).to(torch.float32) / 127.5 - 1
target_images = target_images[:, None, :, :]

ROOT_DIR = f"{WORKING_DIR}/models/others/"
model_dirs = {
    "e10-pre": "e10-pre/new/00000-stylegan2-pre-training-gpus4-batch112-gamma10",
    "e11-pre": "e11-pre/00002-stylegan2-pre-training-gpus4-batch112-gamma10",
    "e12-pre": "e12-pre/00001-stylegan2-pre-training-gpus4-batch112-gamma10",
    "e13-pre": "e13-pre/00002-stylegan2-pre-training-gpus4-batch112-gamma10",
    "e13": "e13/00001-stylegan2-trainingset2-gpus4-batch96-gamma10",
    "e14-pre": "e14-pre/00001-stylegan2-pre-training-gpus4-batch112-gamma10",
    "e15-pre": "e15-pre/00000-stylegan2-pre-training-gpus4-batch112-gamma10"
}

for model_label in model_dirs:
    model_dir = os.path.join(ROOT_DIR, model_dirs[model_label], "network-snapshot-000252.pkl")
    reconstructed_dir = os.path.join(OUT_DIR, "reconstructions_none", model_label)
    os.makedirs(reconstructed_dir, exist_ok=True)
    with dnnlib.util.open_url(model_dir) as fp:
        models = legacy.load_network_pkl(fp)
        E = models['E_ema'].to(device)
        E.b4.get_mu.mbstd = MinibatchStdLayer(4).eval()
        E.b4.get_logvar.mbstd = MinibatchStdLayer(4).eval()
        G = models['G_ema'].to(device)  # type: ignore

    z, logvar = E.mu_var(target_images, None)
    for random_seed in [6, 26]:
        reset_noise_const(G, random_seed)
        reconstructed_images = G(z, None, noise_mode='const')
        reconstructed_images = (reconstructed_images + 1) * 127.5
        reconstructed_images = reconstructed_images.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8).cpu().numpy()[:,
        :, :,
        0]
        for reconstructed_image, img_name in zip(reconstructed_images, img_names):
            cv2.imwrite(os.path.join(reconstructed_dir, f"{random_seed - 6}_{img_name}"), reconstructed_image)
        # row1 = np.hstack(reconstructed_images[:4])  # First row
        # row2 = np.hstack(reconstructed_images[4:8])  # Second row
        # grid_image = np.vstack([row1, row2])  # Stack rows
        # cv2.imwrite(os.path.join(OUT_DIR, "reconstructions_none", f"{random_seed}_{model_label}.png"), grid_image)

Setting up PyTorch plugin "bias_act_plugin"... 

C:\Users\zhang\.conda\envs\py3_9\lib\site-packages\torch\utils\cpp_extension.py:446: UserWarning: Error checking compiler version for /usr/bin/g++-9: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
C:\Users\zhang\.conda\envs\py3_9\lib\site-packages\torch\utils\cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [3]:
model_dirs = {
    "e13-post": "e13-post/from302kimgs/00000-stylegan2-trainingset2-gpus4-batch112-gamma10",
    "e13-scratch": "e13-scratch/00000-stylegan2-trainingset2-gpus4-batch112-gamma10",
}

for model_label in model_dirs:
    model_dir = os.path.join(ROOT_DIR, model_dirs[model_label], "network-snapshot-001512.pkl")
    reconstructed_dir = os.path.join(OUT_DIR, "reconstructions_none", model_label)
    os.makedirs(reconstructed_dir, exist_ok=True)
    with dnnlib.util.open_url(model_dir) as fp:
        models = legacy.load_network_pkl(fp)
        E = models['E_ema'].to(device)
        E.b4.get_mu.mbstd = MinibatchStdLayer(4).eval()
        E.b4.get_logvar.mbstd = MinibatchStdLayer(4).eval()
        G = models['G_ema'].to(device)  # type: ignore

    z, logvar = E.mu_var(target_images, None)
    for random_seed in [6, 26]:
        reset_noise_const(G, random_seed)
        reconstructed_images = G(z, None, noise_mode='const')
        reconstructed_images = (reconstructed_images + 1) * 127.5
        reconstructed_images = reconstructed_images.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8).cpu().numpy()[:,
        :, :,
        0]
        for reconstructed_image, img_name in zip(reconstructed_images, img_names):
            cv2.imwrite(os.path.join(reconstructed_dir, f"{random_seed - 6}_{img_name}"), reconstructed_image)
        # row1 = np.hstack(reconstructed_images[:4])  # First row
        # row2 = np.hstack(reconstructed_images[4:])  # Second row
        # grid_image = np.vstack([row1, row2])  # Stack rows
        # cv2.imwrite(os.path.join(OUT_DIR, "reconstructions_none", f"{model_label}.png"), grid_image)